## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as m
from transforms import *
from torchvision.transforms import Compose
from torchsummary import summary
from repeat_image_dataset import RepeatImageDataset
from random_caption_dataset import RandomCaptionDataset

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PyTorch version: 1.7.1
GPU found :)


In [2]:
IMAGE_SIZE = 224

## Datas section

In [11]:
transforms = Compose([Rescale(256), RandomCrop(IMAGE_SIZE), ToTensor(), Normalize()])

train_repeat_dataset = RepeatImageDataset('./flickr8k/images/train/', transform=transforms)
train_random_dataset = RandomCaptionDataset('./flickr8k/images/train/', transform=transforms)

In [12]:
im = train_repeat_dataset[0]['image'].reshape((1,3,224,224)).to(device, dtype=torch.float)
im

tensor([[[[-0.1457, -0.2434, -0.2860,  ..., -1.6554, -0.8509, -0.3689],
          [-0.1634, -0.1988, -0.2531,  ..., -1.6453, -2.0156, -1.8454],
          [-0.1589, -0.1513, -0.2910,  ..., -0.5445, -0.0285, -1.9745],
          ...,
          [-1.9259, -2.0229, -2.0571,  ..., -0.1126,  0.4675, -1.2379],
          [-1.8919, -1.9941, -2.0218,  ...,  0.4873, -0.3833, -1.4834],
          [-1.9452, -2.0307, -2.0272,  ..., -0.5644, -1.8123, -1.4203]],

         [[ 0.0995,  0.0669, -0.0481,  ..., -1.0403, -0.1754, -0.2705],
          [ 0.1018,  0.0872, -0.0275,  ..., -1.4650, -1.7818, -1.6167],
          [ 0.1131,  0.1071, -0.0432,  ..., -0.5462, -0.7868, -1.9538],
          ...,
          [-1.8571, -1.9124, -1.9438,  ..., -0.2663, -0.3753, -1.2945],
          [-1.8540, -1.9446, -1.9432,  ...,  0.0872, -1.2088, -1.6308],
          [-1.8610, -1.9466, -1.9336,  ..., -1.5683, -1.6659, -1.3400]],

         [[-0.0488, -0.1192, -0.0272,  ..., -1.6421, -1.0877, -0.5378],
          [-0.0382, -0.0795, -

## Model section

In [5]:
# retrieve pretrained model for features extraction
resnet = m.resnet18(pretrained=True).to(device)
#resnet

In [6]:
# Keep only the feature extraction layers of the model
features = nn.Sequential(*(list(resnet.children())[:-2]), nn.Flatten())
summary(features, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

## Build LSTM + Embedding

In [7]:
vocab_size = 9631

class LSTMCaptioning(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        self.lstm = nn.LSTM(input_size, hidden_size)
        
    def forward(self, inputs):
         
        # Get hidden states for each t (out) , and latest one (h = (ht, ct))
        out, ht = self.lstm(inputs.view(len(inputs), 1, -1))
        
        # Compute probability distribution over all words for this t
        pt = F.softmax(ht, dim=1)
                           
        return out, ht, pt

embedding = nn.Embedding(vocab_size, 25088)

## Train model

In [8]:
input_size = (3, 224, 224)
hideen_size = ...

model = LSTMCaptioning(input_size, hidden_size)

num_epoch = 10
loss_function = nn.NLLLoss()

for epoch in range(num_epoch):
    for image, caption in dataset:
        
        # Reset grad
        model.zero_grad()
        
        # Get the input image embedding 
        image_embedding = features(image)
        
        # Forward pass for t=-1: image
        out, hidden, probs = model(image_embedding)
        
        # Forward pass for t>=0: words
        for word in caption:
            word_vector = preprocess(word)
            word_embedding = embedding(word_vector)
        
        # Compute loss and backprop
        loss = loss_function(out, targets)
        loss.backward()
        optimizer.step()
        

NameError: name 'hidden_size' is not defined

In [ ]:
# Save model 
torch.save(model.state_dict(), './models/')